In [1]:
import pandas as pd
import numpy as np

save_date = "7_15_24"
df = pd.read_csv(f"/mnt/Helium/neeraj/mayo-outcome-prediction/all_experiments_from_scratch_summer2024/finalized_dys0_ec_psd_tensor_{save_date}.csv")
df

,mrn,mrn_datetime,age,grade,num_ec_epochs,num_n2_epochs,num_n3_epochs,complaint,eeg_report,Mayo ID,Last Name (First 4 Letters),Can I use medical records for this patient?
0,12032438,12032438_20180608T184826,26.256528,0,6,6,0,Seizure HCC,Clinical Interpretation: Normal EEG during wak...,12032438,Cohe,Use
1,6203122,6203122_20171031T181711,65.496211,0,6,6,0,Amnesia Transient Global Stroke CVA,nan Electroencephalography Fi...,6203122,Bjor,Use
2,6310079,6310079_20120329T163721,61.925981,0,6,6,0,Presyncope,nan 29-Mar-2012 Electroencephalogr...,6310079,Cook,Use
3,3479109,3479109_20171221T190955,74.539518,0,6,6,1,Seizure SZ NOS,nan Electroencephalography Fi...,3479109,Wege,Use
4,6393149,6393149_20200601T155344,22.242757,0,6,6,0,Complaint Memory Spells Undifferentiated,Clinical interpretation: The EEG is normal dur...,6393149,Vaug,Use
...,...,...,...,...,...,...,...,...,...,...,...,...
6237,12869653,12869653_20201210T180304,58.306468,0,6,6,3,Fatigue Stroke HCC,Clinical Interpretation: The short-term video ...,12869653,McEl,Use
6238,3280644,3280644_20180727T180150,76.313682,0,6,6,1,Impairment Cognitive Mild Major Neurocognitive...,Clinical Interpretation: Normal EEG during wak...,3280644,Deye,Use
6239,10597353,10597353_20170906T123614,31.023224,0,6,6,6,Seizure SZ Disorder,nan Electroencephalography Fi...,10597353,Nels,Use
6240,12163503,12163503_20190516T155056,56.934776,0,6,6,1,Stroke HCC Anticoagulant Therapy,CLINICAL INTERPRETATION Normal EEG during wake...,12163503,Mich,Use


In [2]:
from sklearn.model_selection import train_test_split

def get_subject_train_val_split(index_df, train_size, test_size, random_seed):

	unique_subjects = index_df["mrn"].unique().tolist()

	train_subjects, val_subjects = train_test_split(unique_subjects, test_size=test_size, shuffle=True, random_state=random_seed)

	assert len(set(train_subjects).intersection(set(val_subjects))) == 0
	assert set(train_subjects).isdisjoint(val_subjects)

	train_df = index_df[index_df['mrn'].isin(train_subjects)]
	val_df = index_df[index_df['mrn'].isin(val_subjects)]

	return train_df, val_df

In [3]:
RANDOM_SEED = 8765

# get subject-level train/val splits for the training domains
train_and_val_df, test_df = get_subject_train_val_split(
    index_df=df, 
    train_size=1100, 
    test_size=100,
    random_seed=RANDOM_SEED
)

train_df, val_df = get_subject_train_val_split(
    index_df=train_and_val_df, 
    train_size=1000, 
    test_size=100, 
    random_seed=RANDOM_SEED
)

In [4]:
date = "5_12_25"

train_df.reset_index(drop=True, inplace=True)
train_df.insert(0, column="global_idx", value=train_df.index.values)
train_df.to_csv(f"./SSLProbe_train_index_{date}.csv", index=False)
print("train df shape:", train_df.shape)

val_df.reset_index(drop=True, inplace=True)
val_df.insert(0, column="global_idx", value=val_df.index.values)
val_df.to_csv(f"./SSLProbe_val_index_{date}.csv", index=False)
print("val df shape:", val_df.shape)

test_df.reset_index(drop=True, inplace=True)
test_df.insert(0, column="global_idx", value=test_df.index.values)
test_df.to_csv(f"./SSLProbe_test_index_{date}.csv", index=False)
print("test df shape:", test_df.shape)

# NOTE: second sanity check of subject overlap between splits
train_subjects = list(np.unique(train_df['mrn'].values))
val_subjects = list(np.unique(val_df['mrn'].values))
test_subjects = list(np.unique(test_df['mrn'].values))

# print(train_subjects, "\n------------")
# print(val_subjects, "\n------------")
# print(test_subjects, "\n------------")

assert set(train_subjects).isdisjoint(val_subjects)
assert set(train_subjects).isdisjoint(test_subjects)
assert set(val_subjects).isdisjoint(test_subjects)

# # print balance of target class in each set
# print("train targets:", train_df["eeg_dys_grade"].value_counts().to_dict())
# print("val targets:", val_df["eeg_dys_grade"].value_counts().to_dict())
# print("test targets:", test_df["eeg_dys_grade"].value_counts().to_dict())

print("*** ALL DONE! ***")

train df shape: (6034, 13)
val df shape: (103, 13)
test df shape: (105, 13)
*** ALL DONE! ***


In [5]:
len(train_subjects), len(val_subjects), len(test_subjects)

(5765, 100, 100)

In [8]:
val_df

,global_idx,mrn,mrn_datetime,age,grade,num_ec_epochs,num_n2_epochs,num_n3_epochs,complaint,eeg_report,Mayo ID,Last Name (First 4 Letters),Can I use medical records for this patient?
0,0,7189427,7189427_20190617T121516,28.055333,0,6,6,0,Seizure Disorder HCC,Clinical Interpretation: Normal EEG during wak...,7189427,Canc,Use
1,1,3333829,3333829_20191120T193733,71.021308,0,6,6,0,Migraine Headache With Aura,Clinical Interpretation: Normal EEG during wak...,3333829,Edwa,Use
2,2,12678604,12678604_20210420T121930,19.488422,0,6,6,6,Seizure HCC,Clinical Interpretation: Normal EEG during wak...,12678604,Slin,Use
3,3,9084267,9084267_20170721T184516,46.492399,0,6,6,0,Spells Undifferentiated Slurred Speech,nan Electroencephalography Fi...,9084267,Jack,Use
4,4,3110784,3110784_20170913T184513,48.003724,0,6,6,0,Loss Memory NOS,nan Electroencephalography Fi...,3110784,Zwie,Use
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,98,4410499,4410499_20180306T152011,77.036489,0,6,6,0,Spells Undifferentiated,nan Electroencephalography Fi...,4410499,Frit,Use
99,99,4678022,4678022_20170606T180103,72.776306,0,6,6,0,Seizure SZ Disorder,nan Electroencephalography Fi...,4678022,Thil,Use
100,100,9138208,9138208_20170615T172625,18.897034,0,6,6,6,Spells Atypical Seizure SZ,nan Electroencephalography Fi...,9138208,Case,Use
101,101,4525679,4525679_20180604T193659,50.114650,0,6,6,0,Epilepsy Seizure Not Intractable Without Statu...,Clinical Interpretation: The short-term video ...,4525679,Mank,Use
